In [1]:
import numpy as np
import re
from astropy.io import fits
from astropy.stats import sigma_clipped_stats, SigmaClip
from photutils.segmentation import detect_threshold, detect_sources
from photutils.utils import circular_footprint
from photutils import Background2D, SExtractorBackground

/var/folders/p7/drxzchtj4yb641v79lt0tjyh0000gn/T/ipykernel_74422/3187025943.py:7: DeprecationWarning: `photutils.Background2D` is a deprecated alias for `photutils.background.Background2D` and will be removed in the future. Instead, please use `from photutils.background import Background2D` to silence this warning.
  from photutils import Background2D, SExtractorBackground
/var/folders/p7/drxzchtj4yb641v79lt0tjyh0000gn/T/ipykernel_74422/3187025943.py:7: DeprecationWarning: `photutils.SExtractorBackground` is a deprecated alias for `photutils.background.SExtractorBackground` and will be removed in the future. Instead, please use `from photutils.background import SExtractorBackground` to silence this warning.
  from photutils import Background2D, SExtractorBackground


## No Source Masking

#### (25, 25) box size

In [48]:
box_size = (25, 25)

In [49]:
sigma_clip = SigmaClip(sigma=3.0)
bkg_estimator = SExtractorBackground(sigma_clip)

In [50]:
fname = "science_reduced_img.W0019-1046.60201.R_SPECIAL.chip1.1.crz.fits"
h = fits.open("crz/"+fname)
mask  = fits.getdata("masks/"+re.sub(".crz.fits",".mask.fits" ,fname))
emask = fits.getdata("masks/"+re.sub(".crz.fits",".emask.fits",fname))
omask = fits.getdata("masks/"+re.sub(".crz.fits",".omask.fits",fname))

In [51]:
ebkg = Background2D(h[0].data, box_size , filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=emask.astype(bool))
obkg = Background2D(h[0].data, box_size, filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=omask.astype(bool))
h[0].data -= ebkg.background*(1-emask) + obkg.background*(1-omask)

In [52]:
h.writeto("test_bkg1.fits",overwrite=True)

In [53]:
ebkg = Background2D(h[0].data, box_size , filter_size=(15,15), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=emask.astype(bool), exclude_percentile=50.0)
obkg = Background2D(h[0].data, box_size, filter_size=(15,15), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=omask.astype(bool), exclude_percentile=50.0)
h[0].data -= ebkg.background*(1-emask) + obkg.background*(1-omask)

In [54]:
h.writeto("test_bkg1.f15.fits",overwrite=True)

#### (50, 50) box_size

In [7]:
box_size = (50, 50)

In [8]:
fname = "science_reduced_img.W0019-1046.60201.R_SPECIAL.chip1.1.crz.fits"
h = fits.open("crz/"+fname)
mask  = fits.getdata("masks/"+re.sub(".crz.fits",".mask.fits" ,fname))
emask = fits.getdata("masks/"+re.sub(".crz.fits",".emask.fits",fname))
omask = fits.getdata("masks/"+re.sub(".crz.fits",".omask.fits",fname))

In [9]:
ebkg = Background2D(h[0].data, box_size , filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=emask.astype(bool))
obkg = Background2D(h[0].data, box_size, filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=omask.astype(bool))
h[0].data -= ebkg.background*(1-emask) + obkg.background*(1-omask)

In [10]:
h.writeto("test_bkg2.fits",overwrite=True)

## With source masking

#### (25, 25) box_size

In [11]:
box_size = (25, 25)

In [12]:
fname = "science_reduced_img.W0019-1046.60201.R_SPECIAL.chip1.1.crz.fits"
h = fits.open("crz/"+fname)
mask  = fits.getdata("masks/"+re.sub(".crz.fits",".mask.fits" ,fname))
emask = fits.getdata("masks/"+re.sub(".crz.fits",".emask.fits",fname))
omask = fits.getdata("masks/"+re.sub(".crz.fits",".omask.fits",fname))

In [13]:
sigma_clip = SigmaClip(sigma=3.0, maxiters=10)
threshold = detect_threshold(h[0].data, nsigma=1.0, sigma_clip=sigma_clip)
segment_img = detect_sources(h[0].data, threshold, npixels=5)
footprint = circular_footprint(radius=10)
source_mask = segment_img.make_source_mask(footprint=footprint)

In [14]:
ebkg = Background2D(h[0].data, box_size , filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=emask.astype(bool), mask=source_mask)
obkg = Background2D(h[0].data, box_size, filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=omask.astype(bool), mask=source_mask)
h[0].data -= ebkg.background*(1-emask) + obkg.background*(1-omask)

In [15]:
h.writeto("test_bkg3.fits",overwrite=True)

In [16]:
fits.writeto("test_bkg3.smask.fits", source_mask.astype(int), overwrite=True)

#### (50, 50) box_size

In [17]:
box_size = (50,50)

In [18]:
fname = "science_reduced_img.W0019-1046.60201.R_SPECIAL.chip1.1.crz.fits"
h = fits.open("crz/"+fname)
mask  = fits.getdata("masks/"+re.sub(".crz.fits",".mask.fits" ,fname))
emask = fits.getdata("masks/"+re.sub(".crz.fits",".emask.fits",fname))
omask = fits.getdata("masks/"+re.sub(".crz.fits",".omask.fits",fname))

In [19]:
sigma_clip = SigmaClip(sigma=3.0, maxiters=10)
threshold = detect_threshold(h[0].data, nsigma=1.0, sigma_clip=sigma_clip)
segment_img = detect_sources(h[0].data, threshold, npixels=5)
footprint = circular_footprint(radius=10)
source_mask = segment_img.make_source_mask(footprint=footprint)

In [20]:
ebkg = Background2D(h[0].data, box_size , filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=emask.astype(bool), mask=source_mask)
obkg = Background2D(h[0].data, box_size, filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=omask.astype(bool), mask=source_mask)
h[0].data -= ebkg.background*(1-emask) + obkg.background*(1-omask)

In [21]:
h.writeto("test_bkg4.fits",overwrite=True)

In [22]:
fits.writeto("test_bkg4.smask.fits", source_mask.astype(int), overwrite=True)

## Iterative

In [35]:
box_size = (25, 25)

In [36]:
fname = "science_reduced_img.W0019-1046.60201.R_SPECIAL.chip1.1.crz.fits"
h = fits.open("test_bkg1.fits")
mask  = fits.getdata("masks/"+re.sub(".crz.fits",".mask.fits" ,fname))
emask = fits.getdata("masks/"+re.sub(".crz.fits",".emask.fits",fname))
omask = fits.getdata("masks/"+re.sub(".crz.fits",".omask.fits",fname))

In [37]:
sigma_clip = SigmaClip(sigma=3.0, maxiters=10)
threshold = detect_threshold(h[0].data, nsigma=2.0, sigma_clip=sigma_clip)
segment_img = detect_sources(h[0].data, threshold, npixels=10)
footprint = circular_footprint(radius=10)
source_mask = segment_img.make_source_mask(footprint=footprint)

In [38]:
h = fits.open("crz/"+fname)
ebkg = Background2D(h[0].data, box_size , filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=emask.astype(bool), mask=source_mask)
obkg = Background2D(h[0].data, box_size, filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, coverage_mask=omask.astype(bool), mask=source_mask)
h[0].data -= ebkg.background*(1-emask) + obkg.background*(1-omask)

In [39]:
h.writeto("test_bkg5.fits",overwrite=True)

In [40]:
fits.writeto("test_bkg5.smask.fits", source_mask.astype(int), overwrite=True)